In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Import các thư viện cần thiết

In [0]:
import sys
import time
from scipy.special import expit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import kernel_ridge
from sklearn import svm
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.base import BaseEstimator, TransformerMixin

## 2. Load data

In [3]:
data_df = pd.read_csv('drive/My Drive/Final Project/DataScience/Data/weather.csv')
data_df

,time,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windBearing,cloudCover,uvIndex,visibility
0,1262304000,73.45,75.07,72.69,0.97,1010.3,6.72,71.0,0.50,0.0,NaN
1,1262390400,71.67,72.95,69.81,0.94,1011.0,2.29,41.0,0.44,0.0,4.974
2,1262476800,75.25,76.89,73.39,0.94,1011.6,1.16,30.0,0.75,0.0,4.346
3,1262563200,75.25,76.64,71.61,0.88,1010.9,3.46,71.0,0.75,0.0,6.216
4,1262649600,75.25,76.89,73.39,0.94,1010.1,6.91,120.0,0.44,0.0,6.216
...,...,...,...,...,...,...,...,...,...,...,...
725,1324944000,71.67,72.95,69.81,0.94,1012.0,4.61,0.0,0.75,0.0,5.591
726,1325030400,73.45,75.18,73.39,1.00,1013.7,4.61,11.0,0.75,0.0,3.108
727,1325116800,71.67,72.44,66.21,0.83,1012.8,2.29,11.0,0.44,0.0,6.216
728,1325203200,73.45,74.65,69.81,0.88,1012.4,4.61,0.0,0.44,0.0,6.216


## 3. Tách data thành ba tập train, test và validation với tỉ lệ 6:2:2



In [4]:
# Lấy output và chỉ lấy 729 phần tử sau
y_data = data_df["humidity"].copy()
y_data = y_data[1:]
y_data

1      0.94
2      0.94
3      0.88
4      0.94
5      0.88
       ... 
725    0.94
726    1.00
727    0.83
728    0.88
729    0.94
Name: humidity, Length: 729, dtype: float64

In [5]:
# Xóa dòng cuối cùng trong data_df vì dòng này không có output
X_data = data_df.drop(index=729).copy()
X_data

,time,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windBearing,cloudCover,uvIndex,visibility
0,1262304000,73.45,75.07,72.69,0.97,1010.3,6.72,71.0,0.50,0.0,NaN
1,1262390400,71.67,72.95,69.81,0.94,1011.0,2.29,41.0,0.44,0.0,4.974
2,1262476800,75.25,76.89,73.39,0.94,1011.6,1.16,30.0,0.75,0.0,4.346
3,1262563200,75.25,76.64,71.61,0.88,1010.9,3.46,71.0,0.75,0.0,6.216
4,1262649600,75.25,76.89,73.39,0.94,1010.1,6.91,120.0,0.44,0.0,6.216
...,...,...,...,...,...,...,...,...,...,...,...
724,1324857600,66.27,66.48,60.80,0.83,1013.5,5.74,350.0,0.44,0.0,5.591
725,1324944000,71.67,72.95,69.81,0.94,1012.0,4.61,0.0,0.75,0.0,5.591
726,1325030400,73.45,75.18,73.39,1.00,1013.7,4.61,11.0,0.75,0.0,3.108
727,1325116800,71.67,72.44,66.21,0.83,1012.8,2.29,11.0,0.44,0.0,6.216


In [9]:
# Tách dữ liệu thành 3 tập train : test : validation = 6:2:2
seed = 123
X_data_temp, X_data_val, y_data_temp, y_data_val = train_test_split(X_data, y_data, test_size=0.2, random_state=seed)
X_data_train, X_data_test, y_data_train, y_data_test = train_test_split(X_data_temp, y_data_temp, test_size=0.25, random_state=seed)
print(len(X_data_train), len(y_data_train))
print(len(X_data_val), len(y_data_val))
print(len(X_data_test), len(y_data_test))

437 437
146 146
146 146


## 4. Tiền xử lý dữ liệu

In [38]:
X_data_train

,time,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windBearing,cloudCover,uvIndex,visibility
175,1277424000,80.65,87.67,76.99,0.89,1007.2,5.74,230.0,0.19,1.0,6.216
101,1271030400,82.48,89.61,75.19,0.79,1009.3,5.74,180.0,0.44,1.0,6.216
364,1293753600,73.45,74.91,71.61,0.94,1009.6,2.29,11.0,0.75,0.0,4.974
572,1311724800,77.06,79.15,76.99,1.00,1007.8,6.91,240.0,0.75,1.0,4.346
456,1301702400,78.88,82.18,73.39,0.83,1009.3,2.29,270.0,0.75,1.0,3.108
...,...,...,...,...,...,...,...,...,...,...,...
15,1263600000,69.87,70.21,62.59,0.78,1015.6,5.74,11.0,0.75,0.0,6.216
672,1320364800,75.25,77.17,75.19,1.00,1008.4,4.61,350.0,0.19,1.0,6.216
556,1310342400,78.88,82.55,75.19,0.89,1005.7,2.24,NaN,0.75,1.0,6.216
337,1291420800,73.45,74.65,69.81,0.88,1007.4,6.91,0.0,0.75,0.0,6.216


In [0]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df, y=None):
        tp = X_df.copy()
        time = pd.to_datetime(tp['time'], unit='s')
        day_sr = time.dt.day
        month_sr = time.dt.month
        tp.drop(labels=["visibility", "apparentTemperature", "time"], axis=1, inplace=True)
        tp['day'] = pd.Series(day_sr, index=tp.index)
        tp['month'] = pd.Series(month_sr, index=tp.index) 
        return tp

In [42]:
CAD = ColAdderDropper()
tp = CAD.transform(X_data_train)
tp

,temperature,dewPoint,humidity,pressure,windSpeed,windBearing,cloudCover,uvIndex,day,month
175,80.65,76.99,0.89,1007.2,5.74,230.0,0.19,1.0,25,6
101,82.48,75.19,0.79,1009.3,5.74,180.0,0.44,1.0,12,4
364,73.45,71.61,0.94,1009.6,2.29,11.0,0.75,0.0,31,12
572,77.06,76.99,1.00,1007.8,6.91,240.0,0.75,1.0,27,7
456,78.88,73.39,0.83,1009.3,2.29,270.0,0.75,1.0,2,4
...,...,...,...,...,...,...,...,...,...,...
15,69.87,62.59,0.78,1015.6,5.74,11.0,0.75,0.0,16,1
672,75.25,75.19,1.00,1008.4,4.61,350.0,0.19,1.0,4,11
556,78.88,75.19,0.89,1005.7,2.24,NaN,0.75,1.0,11,7
337,73.45,69.81,0.88,1007.4,6.91,0.0,0.75,0.0,4,12


## 5. Tạo pipeline để huấn luyện model